# Toronto Dwellings Analysis Dashboard

In this notebook, you will compile the visualizations from the previous analysis into functions to create a Panel dashboard.

In [ ]:
# imports
import panel as pn
pn.extension('plotly')
import plotly.express as px
import pandas as pd
import hvplot.pandas
import matplotlib.pyplot as plt
import os
from pathlib import Path
from dotenv import load_dotenv

In [ ]:
# Initialize the Panel Extensions (for Plotly)
import panel as pn
pn.extension("plotly")

In [ ]:
# Read the Mapbox API key
load_dotenv()
map_box_api = os.getenv("mapbox")
px.set_mapbox_access_token(map_box_api)

# Import Data

In [ ]:
# Import the CSVs to Pandas DataFrames
file_path = Path("Data/toronto_neighbourhoods_census_data.csv")
to_data = pd.read_csv(file_path, index_col="year")

file_path = Path("Data/toronto_neighbourhoods_coordinates.csv")
df_neighbourhood_locations = pd.read_csv(file_path)

- - -

## Panel Visualizations

In this section, you will copy the code for each plot type from your analysis notebook and place it into separate functions that Panel can use to create panes for the dashboard. 

These functions will convert the plot object to a Panel pane.

Be sure to include any DataFrame transformation/manipulation code required along with the plotting code.

Return a Panel pane object from each function that can be used to build the dashboard.

Note: Remove any `.show()` lines from the code. We want to return the plots instead of showing them. The Panel dashboard will then display the plots.

### Global available data

In [ ]:
#Getting all data ready

In [ ]:
# Getting the data from the top 10 expensive neighbourhoods
average_house_value_by_neighbourhood = to_data.loc[:, ["neighbourhood","average_house_value"]].groupby(by=["neighbourhood"]).mean()
average_house_value_by_neighbourhood_top10 = average_house_value_by_neighbourhood.sort_values('average_house_value', ascending=False).head(10)
avg_house_price_by_neighbourhood =  to_data.loc[:, ["neighbourhood","average_house_value"]].groupby(by=["neighbourhood", "year"]).mean()

In [ ]:
# Calculate the mean number of dwelling types units per year
dwelling_types_mean = to_data.loc[:, "single_detached_house":"other_house"].groupby(["year"]).mean()
dwelling_types_mean

In [ ]:
# Calculate the sum number of dwelling types units per year (hint: use groupby)

dwelling_types_sum = to_data.loc[:, "single_detached_house":"other_house"].groupby(["year"]).sum()
dwelling_types_sum

In [ ]:
# Calculate the average monthly shelter costs for owned and rented dwellings
shelter_owned_rented_avg = to_data[["shelter_costs_owned","shelter_costs_rented"]].groupby(["year"]).mean()
shelter_owned_rented_avg

In [ ]:
# Calculate the average house value per year
average_house_value = to_data["average_house_value"].groupby(["year"]).mean()
average_house_value

In [ ]:
# Calculating average house value by neighbourhood
average_house_value_by_neighbourhood = to_data.loc[:, ["neighbourhood","average_house_value"]].groupby(by=["neighbourhood"]).mean()
average_house_value_by_neighbourhood_locations = pd.merge(df_neighbourhood_locations,average_house_value_by_neighbourhood, on="neighbourhood")
average_house_value_by_neighbourhood_locations

### Panel Visualization Functions

In [ ]:
# Define Panel visualization functions
def neighbourhood_map():
    """Neighbourhood Map"""
    # Join the average values with the neighbourhood locations

    map = px.scatter_mapbox(average_house_value_by_neighbourhood_locations,
                        title="Average House Price in Toronto",
                        lat="lat", 
                        lon="lon",     
                        color="average_house_value", 
                        size="average_house_value",
                        color_continuous_scale=px.colors.cyclical.IceFire,
                        height=600,
                        size_max=15, 
                        zoom=9)
    return map

def create_bar_chart(data, title, xlabel, ylabel, color):
    """
    Create a barplot based in the data argument.
    """
    barplot = data.hvplot.bar(
    title= title,
    xlabel = xlabel, 
    ylabel = ylabel,
    color = color
    )
    return barplot
    

def create_line_chart(data, title, xlabel, ylabel, color):
    """
    Create a line chart based in the data argument.
    """
    
    lineplot = data.hvplot.line(
    title= title,
    xlabel = xlabel, 
    ylabel = ylabel,
    color = color
    )
    return lineplot 

def average_house_value():
    """Average house values per year."""
    
    fig_avg_house_value = create_line_chart(
        data = average_house_value,
        title = "Average House Value in Toronto",
        xlabel = "Year",
        ylabel = "Avg House Value",
        color = "blue"
        )
    return fig_avg_house_value

def average_value_by_neighbourhood():
    """Average house values by neighbourhood."""
    
    fig_avg_house_value_neigh = average_house_value_by_neighbourhood.hvplot(
        title="Average House Values in Toronto per Neighbourhood", 
        x='year', 
        y='average_house_value', 
        xlabel = 'Year', 
        ylabel = 'Average House Value',
        yformatter="%.0f",
        groupby="neighbourhood",
        widget_location='right_top'
        )
    return fig_avg_house_value_neigh
def number_dwelling_types():
    """Number of dwelling types per year"""
    
    dwelling_types_by_neighbourhood =  to_data.loc[:, "neighbourhood":"other_house"].groupby(by=["neighbourhood", "year"]).mean()
    
    fig_number_dwelling_types = dwelling_types_by_neighbourhood.hvplot.bar(
        title='Yearly Dwelling Unit Types Per Neighbourhood',
        x='year', 
        rot=90,
        xlabel = 'Year', 
        ylabel = 'Dwelling Unit Types',
        yformatter="%.0f",
        groupby="neighbourhood",
        widget_location='right_top',
        height=555
        )
    return fig_number_dwelling_types
def average_house_value_snapshot():
    """Average house value for all Toronto's neighbourhoods per year."""

    avg_house_price_by_neighbourhood_reset = avg_house_price_by_neighbourhood.reset_index()
    
    fig_avg_house_value_snapshot = px.bar(avg_house_price_by_neighbourhood_reset, 
        title="Average House Values in Toronto per Neighbourhood Snapshot", 
        x="neighbourhood", 
        y="average_house_value",
        color="average_house_value",
        facet_row="year",
            labels={
                "average_house_value": "Avg. House Value",
                "neighbourhood": "Neighbourhood",
                },
        height = 900,
        width = 900
        )

    return fig_avg_house_value_snapshot
def top_most_expensive_neighbourhoods():
    """Top 10 most expensive neighbourhoods."""
    
    fig_top10 = average_house_value_by_neighbourhood_top10.hvplot.bar(
        title="Top 10 Most Expensive Neighbourhoods in Toronto",
        x='neighbourhood', 
        y='average_house_value',
        rot=90,
        xlabel = 'Neighbourhood', 
        ylabel = 'Avg House Value',
        yformatter="%.0f",
        height=500,
        width=600
        )
    return fig_top10
def sunburts_cost_analysis():
    """Sunburst chart to conduct a costs analysis of most expensive neighbourhoods in Toronto per year."""
    

## Panel Dashboard

In this section, you will combine all of the plots into a single dashboard view using Panel. Be creative with your dashboard design!

In [ ]:
# Create a Title for the Dashboard
title_row = "# Toronto Real Estate Market Data from 2001 to 2016"


# Define a welcome text
welcome_text = "This dashboard shows visual analysis of historical house values in Toronto."

welcome_column = pn.Column(welcome_text, neighbourhood_map())

yearly_market_analysis = pn.Column(
    title_row,
    "## Dwelling Types in Toronto from 2001 to 2016",
    create_bar_chart(dwelling_types_sum.loc[2001], "Dwelling Types in Toronto in 2001", "2001", "Dwelling Type Units", "blue"),
    create_bar_chart(dwelling_types_sum.loc[2006], "Dwelling Types in Toronto in 2006", "2006", "Dwelling Type Units", "red"),
    create_bar_chart(dwelling_types_sum.loc[2011], "Dwelling Types in Toronto in 2011", "2011", "Dwelling Type Units", "orange"),
    create_bar_chart(dwelling_types_sum.loc[2016], "Dwelling Types in Toronto in 2016", "2016", "Dwelling Type Units", "purple"),
)

shelter_house_costs = pn.Column(
    title_row,
    "## Average Shelter Costs for Owned vs Rented vs Average House Value in Toronto from 2001 to 2016",
    create_line_chart(shelter_owned_rented_avg["shelter_costs_owned"], "Average Monthly Shelter Cost for Owned Dwellings in Toronto", "Year", "Avg Monthly Shelter Costs", "green"),
    create_line_chart(shelter_owned_rented_avg["shelter_costs_rented"], "Average Monthly Shelter Cost for Rented Dwellings in Toronto", "Year", "Avg Monthly Shelter Costs", "red"),
)

neighbourhood_analysis = pn.Column(
    title_row,
    "## Average House Values by Neighbourhood vs Dwelling Types vs in Toronto from 2001 to 2016",
    average_value_by_neighbourhood(),
    average_house_value_snapshot(),
    number_dwelling_types(),   
)

top_expensive_neighbourhoods = pn.Column(
    title_row,
    "## Top 10 Neighbourhoods with Highest Average House Value vs Yearly Shelter Costs in Toronto from 2001 to 2016",
    average_house_value_by_neighbourhood_top10,
        
)

#Creating tab layout for dashboard
dashboard_tabs = pn.Tabs(
    ("Welcome", welcome_column),
    ("Yearly Market Analysis", yearly_market_analysis),
    ("Shelter Costs Vs. Avg. House Value", shelter_house_costs),
    ("Neighbourhood Analysis", neighbourhood_analysis),
    ("Top Expensive Neighbourhoods", top_expensive_neighbourhoods),
)

# Create the main dashboard
dashboard = pn.Row(pn.Column(title_row, dashboard_tabs))

## Serve the Panel Dashboard

In [ ]:
dashboard.servable()

# Debugging

Note: Some of the Plotly express plots may not render in the notebook through the panel functions.

However, you can test each plot by uncommenting the following code

In [ ]:
#neighbourhood_map().show()

In [ ]:
# create_bar_chart(data, title, xlabel, ylabel, color)

# # Bar chart for 2001
# create_bar_chart(df_dwelling_units.loc[2001], "Dwelling Types in Toronto in 2001", "2001", "Dwelling Type Units", "red")

# # Bar chart for 2006
# create_bar_chart(df_dwelling_units.loc[2006], "Dwelling Types in Toronto in 2006", "2006", "Dwelling Type Units", "blue")

# # Bar chart for 2011
# create_bar_chart(df_dwelling_units.loc[2011], "Dwelling Types in Toronto in 2011", "2011", "Dwelling Type Units", "orange")

# # Bar chart for 2016
# create_bar_chart(df_dwelling_units.loc[2016], "Dwelling Types in Toronto in 2016", "2016", "Dwelling Type Units", "magenta")

In [ ]:
# create_line_chart(data, title, xlabel, ylabel, color)

# # Line chart for owned dwellings
# create_line_chart(df_avg_costs["shelter_costs_owned"], "Average Monthly Shelter Cost for Owned Dwellings in Toronto", "Year", "Avg Monthly Shelter Costs", "blue")

# # Line chart for rented dwellings
# create_line_chart(df_avg_costs["shelter_costs_rented"], "Average Monthly Shelter Cost for Rented Dwellings in Toronto", "Year", "Avg Monthly Shelter Costs", "orange")

In [ ]:
# average_house_value()

In [ ]:
# average_value_by_neighbourhood()

In [ ]:
# number_dwelling_types()

In [ ]:
# average_house_value_snapshot()

In [ ]:
# top_most_expensive_neighbourhoods()

In [ ]:
# sunburts_cost_analysis()